# Introduction to the GenePattern Notebook

Alexander T. Wenzel<br>atwenzel@ucsd.edu

In this notebook, we'll be reviewing tools for profiling an RNA-sequencing (or RNA-seq) experiment. We will first quantify transcript abundance in our samples using the pseudoaligner [Salmon](https://salmon.readthedocs.io/en/latest/salmon.html) and aggregate to gene-level count estimates using [tximport](https://bioconductor.org/packages/release/bioc/html/tximport.html). Using these counts, we will look for differentially expressed genes in our perturbation samples relative to our control samples using [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) and visualize the results using the DifferentialExpressionViewer GenePattern module. Finally, we will test for up- or down-regulated pathways and processes using [Gene Set Enrichment Analysis (GSEA)](http://www.gsea-msigdb.org/gsea/index.jsp).

The data for this example are 6 samples from an [experiment](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE119088) testing the effects of an inhibitor of *EZH2* on a variety of cancer cell lines. Here we will focus on the U2932 Diffuse Large B Cell Lymphoma data.

# Activate demo mode

GenePattern modules run "headlessly" in the Cloud. This enhances the ability of the Notebook environment to serve as an encapsulation of the entire scientific workflow, since analysis components that require larger amounts of time or memory can be sent off to appropriately provisioned cloud servers without relying on the continuity of the notebook kernel. This means that, for example, you can start a sequencing alignment job, turn off your notebook, open the notebook hours later, and view the results of the alignment.

Some of the modules in this notebook will take several minutes to hours to run. Therefore, we recommend that you activate "demo mode" for the duration of this workshop. This means that, when you click "Run" on a module, the notebook will first check if there are example job results that match the input parameters, and, if they exist, will immediately return those results to the notebook. This lets us keep to the workshop schedule while keeping the changes to the user experience minimal. If you want to actually run any of these modules with different parameters or inputs, or to just test out running modules, then **do not** click on "Run" in the interface below this cell. If you have demo mode already activated and want to turn it off, then click `Kernel ->  Restart`.

In [7]:
import nbtools
import demo_mode
from IPython.display import display, HTML

## Alex
%load_ext autoreload 
## Alex
%autoreload 2  

@nbtools.build_ui(parameters={'output_var': {'hide': True}})
def activate_demo_mode():
    """Enable demo mode for this notebook"""
    
    # Any job you enable as a demo job will need to be listed here, 
    # along with the module name and any parameters you want matched.
    # Parameters that are not listed will not be matched. You can 
    # list the same module multiple times, assuming you use different 
    # parameter match sets.
    
    """
    demo_mode.set_demo_jobs([
        {
            'name': 'Conos.Preprocess',
            'job': 389201,                                # Make sure to set the permissions of any demo job to 'public'
            'params': {
                'knn': '40',    # Demo mode should be able to match between int and string, but when in doubt use string
                'perplexity': 50
            }
        }, 
        {
            'name': 'Conos.Cluster',
            'job': 389530,
            'params': {
                'conos_object': 'conos_preprocess_output.rds',   # For file parameters, just list the file name, not URL
                'runleiden': 'True'
            }
        }
    ])
    """
    
    demo_mode.set_demo_jobs([
        {
            "name": "Salmon.Quant",
            #"job": 82034,
            "job": 82039,
            "params": {
                #"Reads": "Reads.list.txt",
                "Reads": [
                    "SRR7758650-DMSO_R1.fastq.gz",
                    "SRR7758650-DMSO_R2.fastq.gz",
                    "SRR7758651-DMSO_R1.fastq.gz",
                    "SRR7758651-DMSO_R2.fastq.gz",
                    "SRR7758652-DMSO_R1.fastq.gz",
                    "SRR7758652-DMSO_R2.fastq.gz",
                    "SRR7758653-EPZ6438_R1.fastq.gz",
                    "SRR7758653-EPZ6438_R2.fastq.gz",
                    "SRR7758654-EPZ6438_R1.fastq.gz",
                    "SRR7758654-EPZ6438_R2.fastq.gz",
                    "SRR7758655-EPZ6438_R1.fastq.gz",
                    "SRR7758655-EPZ6438_R2.fastq.gz"
                ]
            }
        },
        {
            "name": "tximport.DESeq2.Normalize",
            "job": 82042,
            "params": {
                "Quantifications": [
                    "SRR7758650-DMSO.quant.sf",
                    "SRR7758651-DMSO.quant.sf",
                    "SRR7758652-DMSO.quant.sf",
                    "SRR7758653-EPZ6438.quant.sf",
                    "SRR7758654-EPZ6438.quant.sf",
                    "SRR7758655-EPZ6438.quant.sf"
                ]
            }
        }
    ])
    
    # To activate demo mode, just call activate(). This example wraps 
    # the activation call behind a UI Builder cell, but you could have 
    # it called in different ways.

    demo_mode.activate()
    display(HTML('<div class="alert alert-success">Demo mode activated</div>'))
    
    # The code in this call has been left expanded for tutorial purposes

UIBuilder(description='Enable demo mode for this notebook', function_import='nbtools.tool(id="activate_demo_mo…

# Input data

These URLs point to each of the input files, which are paired `fastq` files for a total of 6 samples. You don't have to download these files, they can be passed the first module using these URLs.

<div class="well well-sm">
    <b>SRR7758650-DMSO_R1.fastq.gz</b>: <a href=https://beta.genepattern.org/gp/users/atwenzel2/SRR7758650-DMSO_R1.fastq.gz>https://beta.genepattern.org/gp/users/atwenzel2/SRR7758650-DMSO_R1.fastq.gz</a>
    <br>
    <b>SRR7758650-DMSO_R2.fastq.gz</b>: <a href=https://beta.genepattern.org/gp/users/atwenzel2/SRR7758650-DMSO_R2.fastq.gz>https://beta.genepattern.org/gp/users/atwenzel2/SRR7758650-DMSO_R2.fastq.gz</a>
    <br>
    <br>
    <b>SRR7758651-DMSO_R1.fastq.gz</b>: <a href=https://beta.genepattern.org/gp/users/atwenzel2/SRR7758651-DMSO_R1.fastq.gz>https://beta.genepattern.org/gp/users/atwenzel2/SRR7758651-DMSO_R1.fastq.gz</a>
    <br>
    <b>SRR7758651-DMSO_R2.fastq.gz</b>: <a href=https://beta.genepattern.org/gp/users/atwenzel2/SRR7758651-DMSO_R2.fastq.gz>https://beta.genepattern.org/gp/users/atwenzel2/SRR7758651-DMSO_R2.fastq.gz</a>
    <br>
    <br>
    <b>SRR7758652-DMSO_R1.fastq.gz</b>: <a href=https://beta.genepattern.org/gp/users/atwenzel2/SRR7758652-DMSO_R1.fastq.gz>https://beta.genepattern.org/gp/users/atwenzel2/SRR7758652-DMSO_R1.fastq.gz</a>
    <br>
    <b>SRR7758652-DMSO_R2.fastq.gz</b>: <a href=https://beta.genepattern.org/gp/users/atwenzel2/SRR7758652-DMSO_R2.fastq.gz>https://beta.genepattern.org/gp/users/atwenzel2/SRR7758652-DMSO_R2.fastq.gz</a>
    <br>
    <br>
    <b>SRR7758653-EPZ6438_R1.fastq.gz</b>: <a href=https://beta.genepattern.org/gp/users/atwenzel2/SRR7758653-EPZ6438_R1.fastq.gz>https://beta.genepattern.org/gp/users/atwenzel2/SRR7758653-EPZ6438_R1.fastq.gz</a>
    <br>
    <b>SRR7758653-EPZ6438_R2.fastq.gz</b>: <a href=https://beta.genepattern.org/gp/users/atwenzel2/SRR7758653-EPZ6438_R2.fastq.gz>https://beta.genepattern.org/gp/users/atwenzel2/SRR7758653-EPZ6438_R2.fastq.gz</a>
    <br>
    <br>
    <b>SRR7758654-EPZ6438_R1.fastq.gz</b>: <a href=https://beta.genepattern.org/gp/users/atwenzel2/SRR7758654-EPZ6438_R1.fastq.gz>https://beta.genepattern.org/gp/users/atwenzel2/SRR7758654-EPZ6438_R1.fastq.gz</a>
    <br>
    <b>SRR7758654-EPZ6438_R2.fastq.gz</b>: <a href=https://beta.genepattern.org/gp/users/atwenzel2/SRR7758654-EPZ6438_R2.fastq.gz>https://beta.genepattern.org/gp/users/atwenzel2/SRR7758654-EPZ6438_R2.fastq.gz</a>
    <br>
    <br>
    <b>SRR7758655-EPZ6438_R1.fastq.gz</b>: <a href=https://beta.genepattern.org/gp/users/atwenzel2/SRR7758655-EPZ6438_R1.fastq.gz>https://beta.genepattern.org/gp/users/atwenzel2/SRR7758655-EPZ6438_R1.fastq.gz</a>
    <br>
    <b>SRR7758655-EPZ6438_R2.fastq.gz</b>: <a href=https://beta.genepattern.org/gp/users/atwenzel2/SRR7758655-EPZ6438_R2.fastq.gz>https://beta.genepattern.org/gp/users/atwenzel2/SRR7758655-EPZ6438_R2.fastq.gz</a>
 
</div>

# Login to GenePattern

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Instructions for inserting a GP login cell.
</div>

In [1]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://beta.genepattern.org/gp", "", ""))

GPAuthWidget()

# Get transcript abundances with Salmon

Here we will run the Salmon.Quant module, the GenePattern module that implements the Salmon pseudoalignment tool, a much faster approach to traditional sequencing alignment, which looks for read segments that match overlapping segments of the transcriptome rather than a base-pair-to-base-pair matching. Salmon.Quant needs a valid Salmon transcriptome index to use as a reference. GenePattern has prebuilt references for Human and Mouse transcriptomes. If you have sequencing data from another organism, you can build your own Salmon index with the Salmon.Indexer GenePattern module.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Insert a Salmon.Quant module.
    <ol>
        <li>Create a new cell below this one using the <b>+</b> button.</li>
        <li>Click on "Cell -> CellType -> GenePattern".</li>
        <li>Search for "Salmon.Quant" and choose the Salmon.Quant module from the list.</li>
        <li>Drag each of the <code>fastq.gz</code> links above to the <code>Reads*</code> parameter field.</li>
        <li>Drag <a href=https://datasets-genepattern-org.s3.amazonaws.com/data/test_data/Salmon/gencode.v37.annotation.k31.salmon_full_decoy_index.tar.gz>this link</a> to the <code>Transcriptome Index*</code> parameter field.</li> 
        <li>Click "Run".</li>
    </ol>
</div>

In [2]:
salmon_quant_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:genepattern.org:module.analysis:00420')
salmon_quant_job_spec = salmon_quant_task.make_job_spec()
salmon_quant_job_spec.set_parameter("Reads", ["https://beta.genepattern.org/gp/users/atwenzel2/SRR7758650-DMSO_R1.fastq.gz", "https://beta.genepattern.org/gp/users/atwenzel2/SRR7758650-DMSO_R2.fastq.gz", "https://beta.genepattern.org/gp/users/atwenzel2/SRR7758651-DMSO_R1.fastq.gz", "https://beta.genepattern.org/gp/users/atwenzel2/SRR7758651-DMSO_R2.fastq.gz", "https://beta.genepattern.org/gp/users/atwenzel2/SRR7758652-DMSO_R1.fastq.gz", "https://beta.genepattern.org/gp/users/atwenzel2/SRR7758652-DMSO_R2.fastq.gz", "https://beta.genepattern.org/gp/users/atwenzel2/SRR7758653-EPZ6438_R1.fastq.gz", "https://beta.genepattern.org/gp/users/atwenzel2/SRR7758653-EPZ6438_R2.fastq.gz", "https://beta.genepattern.org/gp/users/atwenzel2/SRR7758654-EPZ6438_R1.fastq.gz", "https://beta.genepattern.org/gp/users/atwenzel2/SRR7758654-EPZ6438_R2.fastq.gz", "https://beta.genepattern.org/gp/users/atwenzel2/SRR7758655-EPZ6438_R1.fastq.gz", "https://beta.genepattern.org/gp/users/atwenzel2/SRR7758655-EPZ6438_R2.fastq.gz"])
salmon_quant_job_spec.set_parameter("Transcriptome.Index", "https://datasets-genepattern-org.s3.amazonaws.com/data/test_data/Salmon/gencode.v37.annotation.k31.salmon_full_decoy_index.tar.gz")
salmon_quant_job_spec.set_parameter("Library.Type", "A")
salmon_quant_job_spec.set_parameter("Sampling", "BOOT")
salmon_quant_job_spec.set_parameter("seqBias", "true")
salmon_quant_job_spec.set_parameter("gcBias", "true")
salmon_quant_job_spec.set_parameter("posBias", "true")
salmon_quant_job_spec.set_parameter("N.Sampling", "30")
salmon_quant_job_spec.set_parameter("Mimic.Bowtie", "off")
salmon_quant_job_spec.set_parameter("Recover.Orphans", "false")
salmon_quant_job_spec.set_parameter("Hard.Filter", "false")
salmon_quant_job_spec.set_parameter("Allow.Dovetail", "false")
salmon_quant_job_spec.set_parameter("Dump.EQ", "false")
salmon_quant_job_spec.set_parameter("reduce.GC.Memory", "false")
salmon_quant_job_spec.set_parameter("Range.Factorization", "false")
salmon_quant_job_spec.set_parameter("Range.Factorization.Bins", "4")
salmon_quant_job_spec.set_parameter("biasSpeedSamp", "false")
salmon_quant_job_spec.set_parameter("biasSpeedSamp.Factor", "5")
salmon_quant_job_spec.set_parameter("job.memory", "16Gb")
salmon_quant_job_spec.set_parameter("job.queue", "gp-new-beta")
salmon_quant_job_spec.set_parameter("job.cpuCount", "3")
salmon_quant_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(salmon_quant_task)


GPTaskWidget(lsid='urn:lsid:genepattern.org:module.analysis:00420')

# Get gene counts and differentially expressed genes

Now that we have the quantifications of transcript abundance, we can explore the changes in expression in the treated samples relative to the control. To do this, we start by running the tximport.DESeq2.Normalize module, which combines the functionality of the tximport and DESeq2 software tools. tximport aggregates the transcript-level abundance estimates to gene-level estimated counts, which are then appropriate for input to DESeq2, which tests for gene-level differential expression. Finally, this module also gives us the expression matrix transformed according to DESeq2's "median of ratios" normalization which it performs prior to testing for differential expression. We find that this normalization leads to optimal performance for Gene Set Enrichment Analysis (GSEA), which we will revisit later in this notebook.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Insert and run tximport.DESeq2.Normalize
    <ol>
        <li>Create a new cell below this one using the <b>+</b> button.</li>
        <li>Click on "Cell -> CellType -> GenePattern".</li>
        <li>Search for "tximport.DESeq2.Normalize" and choose the tximport.DESeq2.Normalize module.</li>
        <li>Drag each of the <code>quant.sf</code> files from the job results above to, or select each one from the dropdown for the <code>Quantifications*</code> parameter field.</li>
        <li>Click "Upload File" next to the <code>Sample Info</code> parameter field and uploaded the provided <code>2021-11-16_intro-to-gp_tximport-phenotypes.txt</code> file.</li>
        <li>Drag <a href=https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_37/gencode.v37.annotation.gtf.gz>this link</a> to the <code>Transcriptome Database</code> parameter field.</li> 
        <li>Set the <code>Reverse Sign*</code> parameter to "TRUE".</li>
    </ol>
</div>

In [3]:
tximport_deseq2_normalize_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:179')
tximport_deseq2_normalize_job_spec = tximport_deseq2_normalize_task.make_job_spec()
tximport_deseq2_normalize_job_spec.set_parameter("Quantifications", ["https://beta.genepattern.org/gp/jobResults/82039/SRR7758650-DMSO.quant.sf", "https://beta.genepattern.org/gp/jobResults/82039/SRR7758651-DMSO.quant.sf", "https://beta.genepattern.org/gp/jobResults/82039/SRR7758652-DMSO.quant.sf", "https://beta.genepattern.org/gp/jobResults/82039/SRR7758653-EPZ6438.quant.sf", "https://beta.genepattern.org/gp/jobResults/82039/SRR7758654-EPZ6438.quant.sf", "https://beta.genepattern.org/gp/jobResults/82039/SRR7758655-EPZ6438.quant.sf"])
tximport_deseq2_normalize_job_spec.set_parameter("Sample.Info", "https://beta.genepattern.org/gp/users/atwenzel2/tmp/run7081044627909928268.tmp/Sample.Info/1/2021-11-16_intro-to-gp_tximport-phenotypes.txt")
tximport_deseq2_normalize_job_spec.set_parameter("Quant.Type", "SALMON")
tximport_deseq2_normalize_job_spec.set_parameter("Transcriptome.Database", "https://beta.genepattern.org/gp/users/atwenzel2/tmp/run7303381141072436253.tmp/Transcriptome.Database/1/gencode.v37.annotation.gtf.gz")
tximport_deseq2_normalize_job_spec.set_parameter("Output.Normalized.Counts", "TRUE")
tximport_deseq2_normalize_job_spec.set_parameter("Output.TPM", "TRUE")
tximport_deseq2_normalize_job_spec.set_parameter("output.file.base", "Merged.Data")
tximport_deseq2_normalize_job_spec.set_parameter("Reverse.Sign", "TRUE")
tximport_deseq2_normalize_job_spec.set_parameter("Annotate.DEGs", "TRUE")
tximport_deseq2_normalize_job_spec.set_parameter("Split.Identifiers", "TRUE")
tximport_deseq2_normalize_job_spec.set_parameter("Min.Count.Filter", "1")
tximport_deseq2_normalize_job_spec.set_parameter("random.seed", "779948241")
tximport_deseq2_normalize_job_spec.set_parameter("job.memory", "2 Gb")
tximport_deseq2_normalize_job_spec.set_parameter("job.queue", "gp-new-beta")
tximport_deseq2_normalize_job_spec.set_parameter("job.cpuCount", "1")
tximport_deseq2_normalize_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(tximport_deseq2_normalize_task)


GPTaskWidget(lsid='urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:179'…

<div class="alert alert-danger">
<p class="lead"> Error <i class="fa fa-exclamation-circle"></i></p>
    FIX <code>Uncaught TypeError: val.lastIndexOf is not a function</code> WHEN UPLOADING A FILE
</div>

# Visualize differential expression results

Here, we will use the DifferentialExpressionViewer GenePattern module to render a visualization of significantly differentially expressed genes within the notebook. This module expects an input in [ODF](https://www.genepattern.org/file-formats-guide#ODF) format, so we will first run the txt2odf module to convert the results of DESeq2 to this file format.

## Convert DESeq2 output to DifferentialExpressionViewer input

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Insert and run the txt2odf module.
    <ol>
        <li>Create a new cell below this one using the <b>+</b> button.</li>
        <li>Click on "Cell -> CellType -> GenePattern".</li>
        <li>Search for "txt2odf" and choose the txt2odf module.</li>
        <li>Drag the <b>DESEQ2 OUTPUT FILE NAME HERE</b> to the <code>txt file*</code> parameter field.</li>
        <li>Drag the <b>DESEQ2 CORRECTED COUNTS FILE NAME HERE</b> to the <code>gct</code> parameter field.</li>
        <li>Drag the <b>DESEQ2 CLS FILE NAME HERE</b> to the <code>cls</code> parameter field.</li>
        <li>Click "Run".</li>
    </ol>
</div>

In [4]:
txt2odf_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00372')
txt2odf_job_spec = txt2odf_task.make_job_spec()
txt2odf_job_spec.set_parameter("txt_file", "")
txt2odf_job_spec.set_parameter("prune_gct", "False")
txt2odf_job_spec.set_parameter("gct", "")
txt2odf_job_spec.set_parameter("cls", "")
txt2odf_job_spec.set_parameter("job.memory", "2 Gb")
txt2odf_job_spec.set_parameter("job.queue", "gp-new-beta")
txt2odf_job_spec.set_parameter("job.cpuCount", "1")
txt2odf_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(txt2odf_task)


GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00372')

<div class="alert alert-danger">
<p class="lead"> Error <i class="fa fa-exclamation-circle"></i></p>
FIX COLUMN NAMES INCOMPATIBILITY IN tximport.DESeq2.Normalize
</div>

## Generate visualization with DifferentialExpressionViewer

Now that we have our data in the correct format, we can visualize the differentially expressed genes.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Insert and run the DifferentialExpressionViewer module.
    <ol>
        <li>Create a new cell below this one using the <b>+</b> button.</li>
        <li>Click on "Cell -> CellType -> GenePattern".</li> 
        <li>Search for "DifferentialExpressionViewer" and choose the DifferentialExpressionViewer module.</li>
        <li>In the dropdown for <code>differential expression filename*</code>, choose the ODF file from txt2odf</li>
        <li>In the <code>dataset filename*</code>, choose the <code>Merged.Data.Normalized.Counts.gct</code> file from tximport.DESeq2.Normalize</li>
        <li>Click "Run"</li>
    </ol>
</div>

In [5]:
differentialexpressionviewer_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.visualizer:00045')
differentialexpressionviewer_job_spec = differentialexpressionviewer_task.make_job_spec()
differentialexpressionviewer_job_spec.set_parameter("differential.expression.filename", "")
differentialexpressionviewer_job_spec.set_parameter("dataset.filename", "")
differentialexpressionviewer_job_spec.set_parameter("job.memory", "2 Gb")
differentialexpressionviewer_job_spec.set_parameter("job.queue", "gp-new-beta")
differentialexpressionviewer_job_spec.set_parameter("job.cpuCount", "1")
differentialexpressionviewer_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(differentialexpressionviewer_task)


GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.visualizer:00045')

# Find dysregulated pathways with GSEA

In [6]:
gsea_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00072')
gsea_job_spec = gsea_task.make_job_spec()
gsea_job_spec.set_parameter("expression.dataset", "")
gsea_job_spec.set_parameter("gene.sets.database", "")
gsea_job_spec.set_parameter("number.of.permutations", "1000")
gsea_job_spec.set_parameter("phenotype.labels", "")
gsea_job_spec.set_parameter("target.profile", "")
gsea_job_spec.set_parameter("permutation.type", "phenotype")
gsea_job_spec.set_parameter("collapse.dataset", "Collapse")
gsea_job_spec.set_parameter("chip.platform.file", "")
gsea_job_spec.set_parameter("output.file.name", "<expression.dataset_basename>.zip")
gsea_job_spec.set_parameter("scoring.scheme", "weighted")
gsea_job_spec.set_parameter("metric.for.ranking.genes", "Signal2Noise")
gsea_job_spec.set_parameter("gene.list.sorting.mode", "real")
gsea_job_spec.set_parameter("gene.list.ordering.mode", "descending")
gsea_job_spec.set_parameter("max.gene.set.size", "500")
gsea_job_spec.set_parameter("min.gene.set.size", "15")
gsea_job_spec.set_parameter("collapsing.mode.for.probe.sets.with.more.than.one.match", "Max_probe")
gsea_job_spec.set_parameter("normalization.mode", "meandiv")
gsea_job_spec.set_parameter("randomization.mode", "no_balance")
gsea_job_spec.set_parameter("omit.features.with.no.symbol.match", "true")
gsea_job_spec.set_parameter("median.for.class.metrics", "false")
gsea_job_spec.set_parameter("number.of.markers", "100")
gsea_job_spec.set_parameter("random.seed", "timestamp")
gsea_job_spec.set_parameter("save.random.ranked.lists", "false")
gsea_job_spec.set_parameter("selected.gene.sets", "")
gsea_job_spec.set_parameter("dev.mode", "false")
gsea_job_spec.set_parameter("alt.delim", "")
gsea_job_spec.set_parameter("plot.graphs.for.the.top.sets.of.each.phenotype", "20")
gsea_job_spec.set_parameter("make.detailed.gene.set.report", "true")
gsea_job_spec.set_parameter("create.svgs", "false")
gsea_job_spec.set_parameter("create.gcts", "false")
gsea_job_spec.set_parameter("create.zip", "true")
gsea_job_spec.set_parameter("job.memory", "8 Gb")
gsea_job_spec.set_parameter("job.queue", "gp-new-beta")
gsea_job_spec.set_parameter("job.cpuCount", "1")
gsea_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(gsea_task)


GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00072')